In [21]:
import os
import snowflake.connector

In [22]:
csv_file_path = "/Users/yomisaka/Documents/Crime Project/UK_Crime_Data.csv"  # Path to cleaned file
stage_name = "crime_stage" # Staging name
staging_table = "staging_crime_data" # Staging table
main_table = "crime_data" # Main Table

In [23]:
snowflake_config = {
    "user": "EMMANUELS",
    "password": "Hastingsdirect1!",
    "account": "ZRDEGRG-JK37645",
    "warehouse": "SNOWFLAKE_LEARNING_WH",
    "database": "UK_POLICE_DATA",
    "schema": "STREET_LEVEL_CRIME"
}

# Connect to Snowflake
conn = snowflake.connector.connect(**snowflake_config)
cursor = conn.cursor()

# Create internal stage
cursor.execute(f"CREATE OR REPLACE STAGE {stage_name}")

# Upload data to stage
put_command = f"PUT file://{os.path.abspath(csv_file_path)} @{stage_name} OVERWRITE = TRUE"
cursor.execute(put_command)

# Creating staging table
cursor.execute(f"""
CREATE OR REPLACE TABLE {staging_table} (
    crime_id STRING,
    month STRING,
    reported_by STRING,
    falls_within STRING,
    longitude STRING,
    latitude STRING,
    location STRING,
    lsoa_code STRING,
    lsoa_name STRING,
    crime_type STRING,
    last_outcome_category STRING,
    context STRING
)
""")

# Loading data into staging table
cursor.execute(f"""
COPY INTO {staging_table}
FROM @{stage_name}/{os.path.basename(csv_file_path)}
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1 FIELD_OPTIONALLY_ENCLOSED_BY = '"')
""")

# Creating the main table
cursor.execute(f"""
CREATE OR REPLACE TABLE {main_table} (
    crime_id STRING,
    month DATE,
    reported_by STRING,
    falls_within STRING,
    longitude FLOAT,
    latitude FLOAT,
    location STRING,
    lsoa_code STRING,
    lsoa_name STRING,
    crime_type STRING,
    last_outcome_category STRING,
    context STRING
)
""")

# Inserting data from staging to main table
cursor.execute(f"""
INSERT INTO {main_table}
SELECT
    crime_id,
    TO_DATE(month) AS month,
    reported_by,
    falls_within,
    TRY_TO_DOUBLE(longitude),
    TRY_TO_DOUBLE(latitude),
    location,
    lsoa_code,
    lsoa_name,
    crime_type,
    last_outcome_category,
    context
FROM {staging_table}
WHERE crime_id IS NOT NULL
  AND (location IS NOT NULL OR (latitude IS NOT NULL AND longitude IS NOT NULL))
""") # Already done data cleaning prior but just doing another not null check


# Close connection
cursor.close()
conn.close()
